<a href="https://colab.research.google.com/github/hrai/M4-Data-Analysis/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import pandas as pd

url='https://raw.githubusercontent.com/hrai/M4-Data-Analysis/master/data/M4_passages_2018-07.csv'
df = pd.read_csv(url)


In [29]:
df.items

df[:3]

,AssetID,FinancialQtrID,Date,IntervalStart,IntervalEnd,Version,VehicleClass,TollPointID,GantryDirection,GantryLocation,GantryGPSLatitude,GantryGPSLongitude,GantryType,TotalVolume
0,M4,201901,2018-07-01,00:00,00:14,3,Car,TS0101,W,Church St Exit,-33.83,151.0,Mainline,310
1,M4,201901,2018-07-01,00:00,00:14,3,Truck,TS0101,W,Church St Exit,-33.83,151.0,Mainline,4
2,M4,201901,2018-07-01,00:00,00:14,3,Car,TS0102,E,Church St Entry,-33.83,151.0,Mainline,120


In [30]:
df.columns

Index(['AssetID', 'FinancialQtrID', 'Date', 'IntervalStart', 'IntervalEnd',
       'Version', 'VehicleClass', 'TollPointID', 'GantryDirection',
       'GantryLocation', 'GantryGPSLatitude', 'GantryGPSLongitude',
       'GantryType', 'TotalVolume'],
      dtype='object')

## Check for null values



In [31]:
df.isnull().values.any()

False

## Rename column

In [32]:
df.rename(columns={'TotalVolume': 'TotalTrafficVolume', 'VehicleClass':'VehicleType'}, inplace=True)
df.columns

Index(['AssetID', 'FinancialQtrID', 'Date', 'IntervalStart', 'IntervalEnd',
       'Version', 'VehicleType', 'TollPointID', 'GantryDirection',
       'GantryLocation', 'GantryGPSLatitude', 'GantryGPSLongitude',
       'GantryType', 'TotalTrafficVolume'],
      dtype='object')

In [33]:
df.values

array([['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 310],
       ['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 4],
       ['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 120],
       ...,
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Ramp', 24],
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Ramp', 5],
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Mainline + G', 17]],
      dtype=object)

## Checking latitute and longitude against GantryLocation

In [34]:
temp_df=df[['GantryLocation','GantryGPSLatitude','GantryGPSLongitude','TollPointID']].groupby(by=['GantryLocation'])
print(temp_df.first())

                            GantryGPSLatitude  GantryGPSLongitude TollPointID
GantryLocation                                                               
Church St Entry                        -33.83              151.00      TS0102
Church St Exit                         -33.83              151.00      TS0101
Hill Rd Entry                          -33.85              151.05      TS1001
Hill Rd Exit                           -33.85              151.05      TS0901
Homebush Bay Dr (NB) Entry             -33.86              151.07      TS1301
Homebush Bay Dr Entry                  -33.86              151.07      TS1401
Homebush Bay Dr Exit                   -33.86              151.07      TS1201
James Ruse Dr Entry                    -33.83              151.02      TS0201
James Ruse Dr Exit                     -33.83              151.02      TS0202
Silverwater Rd Entry                   -33.84              151.04      TS0601
Silverwater Rd Exit                    -33.84              151.0

In [35]:
print(df['AssetID'].unique())
print(df['Version'].unique())
print(df['FinancialQtrID'].unique())
#print(df['Date'].unique())

['M4']
[3]
[201901]


## Drop unwanted columns like latitude and longitude

In [36]:
df.drop(['GantryGPSLatitude', 'GantryGPSLongitude','AssetID', 'Version','TollPointID','FinancialQtrID'], axis = 1, inplace=True)
df.columns

Index(['Date', 'IntervalStart', 'IntervalEnd', 'VehicleType',
       'GantryDirection', 'GantryLocation', 'GantryType',
       'TotalTrafficVolume'],
      dtype='object')

In [37]:
df['GantryLocation'].unique()

array(['Church St Exit', 'Church St Entry', 'James Ruse Dr Entry',
       'James Ruse Dr Exit', 'Silverwater Rd Exit',
       'Silverwater Rd Entry', 'Hill Rd Exit', 'Hill Rd Entry',
       'Homebush Bay Dr Exit', 'Homebush Bay Dr (NB) Entry',
       'Homebush Bay Dr Entry'], dtype=object)

In [39]:
df['VehicleType'].unique()

array(['Car', 'Truck'], dtype=object)

In [0]:
df['GantryType'].unique()

In [40]:
group_by_gantry_type = df.groupby(by=['GantryType', 'VehicleType'])
group_by_gantry_type.head(2)


,Date,IntervalStart,IntervalEnd,VehicleType,GantryDirection,GantryLocation,GantryType,TotalTrafficVolume
0,2018-07-01,00:00,00:14,Car,W,Church St Exit,Mainline,310
1,2018-07-01,00:00,00:14,Truck,W,Church St Exit,Mainline,4
2,2018-07-01,00:00,00:14,Car,E,Church St Entry,Mainline,120
3,2018-07-01,00:00,00:14,Truck,E,Church St Entry,Mainline,2
4,2018-07-01,00:00,00:14,Car,W,James Ruse Dr Entry,Ramp,23
5,2018-07-01,00:00,00:14,Car,E,James Ruse Dr Exit,Ramp,4
10,2018-07-01,00:00,00:14,Truck,W,Silverwater Rd Entry,Ramp,1
13,2018-07-01,00:00,00:14,Truck,W,Silverwater Rd Exit,Ramp,1
22,2018-07-01,00:00,00:14,Car,W,Homebush Bay Dr Entry,Mainline + G,266
23,2018-07-01,00:00,00:14,Truck,W,Homebush Bay Dr Entry,Mainline + G,3


In [41]:
car_data_avg = group_by_gantry_type.mean()
car_data_count = group_by_gantry_type.count()

print(car_data_avg.head(2))
print(car_data_count)

car_data_count

                        TotalTrafficVolume
GantryType VehicleType                    
Mainline   Car                  411.044388
           Truck                 36.510095
                           Date  ...  TotalTrafficVolume
GantryType   VehicleType         ...                    
Mainline     Car           8651  ...                8651
             Truck         8618  ...                8618
Mainline + G Car           2961  ...                2961
             Truck         2862  ...                2862
Ramp         Car          37943  ...               37943
             Truck        33913  ...               33913

[6 rows x 6 columns]


Date  ...  TotalTrafficVolume
GantryType   VehicleType         ...                    
Mainline     Car           8651  ...                8651
             Truck         8618  ...                8618
Mainline + G Car           2961  ...                2961
             Truck         2862  ...                2862
Ramp         Car          37943  ...               37943
             Truck        33913  ...               33913

[6 rows x 6 columns]